# Making old trained models compatible with lightning

TC: https://wandb.ai/gnn_tracking/gnn_tracking/runs/04b2e3ce/
GC: https://wandb.ai/gnn_tracking/gnn_tracking_gc/runs/7dce6aff
EC: https://wandb.ai/gnn_tracking/gnn_tracking_ec/runs/ddff435e

In [1]:
import json

from gnn_tracking.models.graph_construction import GraphConstructionFCNN

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [30]:
from gnn_tracking.utils.loading import TrackingDataModule
dm = TrackingDataModule(
    train=dict(
        dirs=[
            "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/"
        ],
        # If you run into memory issues, reduce this
    ),
    val=dict(
        dirs=[
            "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/"
        ],
        stop=5
    ),
)

## GC

In [3]:
model = GraphConstructionFCNN(in_dim=14, out_dim=8, depth=6, hidden_dim=512, beta=0.4)


In [14]:
import torch

ckpt = torch.load("/home/kl5675/ray_results/gc-hinge-sq-sq-cells/GCTrainable_7dce6aff_24_val_batch_size=1,adam_amsgrad=False,adam_beta1=0.9000,adam_beta2=0.9990,adam_eps=0.0000,adam_weight_decay=_2023-06-08_13-32-02/checkpoint_000009/checkpoint.pt", map_location="cpu")
state_dct = ckpt["model_state_dict"]
state_dct = {f"_{key}": value for key, value in state_dct.items()}
model.load_state_dict(state_dct)

<All keys matched successfully>

In [27]:
from gnn_tracking.training.ml import MLModule
from functools import partial
from gnn_tracking.metrics.losses import GraphConstructionHingeEmbeddingLoss

lmodel = MLModule(
    model=model,
    loss_fct=GraphConstructionHingeEmbeddingLoss(max_num_neighbors=256, r_emb=1, attr_pt_thld=0.9),
    lw_repulsive=0.001953029788887701,  # loss weight, see below
    optimizer=partial(torch.optim.Adam, lr=1e-3),
)

In [31]:
dm.setup(stage="fit")

[18:18:54] INFO: DataLoader will load 900 graphs (out of 900 available).
[18:18:54] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/data21000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/data21999_s0.pt
[18:18:54] INFO: DataLoader will load 5 graphs (out of 1000 available).
[18:18:54] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/data29004_s0.pt


In [35]:
from pytorch_lightning import Trainer

trainer = Trainer(accelerator="cpu", max_epochs=0)

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3 ...
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [36]:
trainer.fit(lmodel, dm)

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3 ...
  rank_zero_warn(
[18:22:36] INFO: DataLoader will load 900 graphs (out of 900 available).
[18:22:36] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/data21000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/data21999_s0.pt
[18:22:36] INFO: DataLoader will load 5 graphs (out of 1000 available).
[18:22:36] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/da

Sanity Checking: 0it [00:00, ?it/s]

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 28 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 28 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=0` reached.


In [37]:
trainer.save_checkpoint("gc-7dce6aff.ckpt")


In [39]:
lmodel.hparams

"loss_fct":     {'class_path': 'gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss', 'init_args': {'r_emb': 1, 'max_num_neighbors': 256, 'attr_pt_thld': 0.9, 'p_attr': 1, 'p_rep': 1}}
"lw_repulsive": 0.001953029788887701
"model":        {'class_path': 'gnn_tracking.models.graph_construction.GraphConstructionFCNN', 'init_args': {'in_dim': 14, 'hidden_dim': 512, 'out_dim': 8, 'depth': 6, 'beta': 0.4}}
"preproc":      None